In [10]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import os


dataPath = "../dataset/n"


dataGen = ImageDataGenerator(rescale = 1./255,validation_split =0.2)


#dados pra treino
train_generator =dataGen.flow_from_directory(
    dataPath,
    target_size=(28,28),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

#dados pra validação
validation_generator = dataGen.flow_from_directory(
    dataPath,
    target_size=(28, 28),
    batch_size=32,
    class_mode='categorical',  
    subset='validation'
)


Found 6713 images belonging to 15 classes.
Found 1671 images belonging to 15 classes.


In [11]:
model = Sequential()

#camadas da rede

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))


model.add(Dense(15, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


model.summary()

c:\Users\rafae\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 26, 26, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 15)             │         1,935 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 226,255 (883.81 KB)

 Trainable params: 226,255 (883.81 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
#treinamento da rede
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10 
)

#avaliação da rede
loss, accuracy = model.evaluate(validation_generator)
print(f'Acurácia do modelo: {accuracy * 100:.2f}%')







c:\Users\rafae\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 23s 98ms/step - accuracy: 0.1826 - loss: 2.4830 - val_accuracy: 0.6941 - val_loss: 1.1176
Epoch 2/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 342us/step - accuracy: 0.4688 - loss: 1.5217 - val_accuracy: 0.5714 - val_loss: 1.4091
Epoch 3/10
  1/209 ━━━━━━━━━━━━━━━━━━━━ 26s 126ms/step - accuracy: 0.6250 - loss: 1.2585

c:\Users\rafae\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


209/209 ━━━━━━━━━━━━━━━━━━━━ 15s 71ms/step - accuracy: 0.6104 - loss: 1.2267 - val_accuracy: 0.8041 - val_loss: 0.6648
Epoch 4/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 203us/step - accuracy: 0.8125 - loss: 1.0070 - val_accuracy: 0.8571 - val_loss: 0.4149
Epoch 5/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 13s 60ms/step - accuracy: 0.7210 - loss: 0.8464 - val_accuracy: 0.8299 - val_loss: 0.5508
Epoch 6/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 156us/step - accuracy: 0.7500 - loss: 0.6799 - val_accuracy: 0.8571 - val_loss: 0.5960
Epoch 7/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 13s 61ms/step - accuracy: 0.7730 - loss: 0.7011 - val_accuracy: 0.8828 - val_loss: 0.4141
Epoch 8/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 167us/step - accuracy: 0.7500 - loss: 0.5625 - val_accuracy: 0.8571 - val_loss: 0.7305
Epoch 9/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 12s 58ms/step - accuracy: 0.7968 - loss: 0.6148 - val_accuracy: 0.8978 - val_loss: 0.3428
Epoch 10/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 173us/step - accuracy: 0.8438 - loss: 0.5881 - val_accura

In [39]:

img_dir = '../test/'

# caminho das imagens
img_paths = [os.path.join(img_dir, img_name) for img_name in os.listdir(img_dir)] 


#array para guardar valores preditos 
predictions = []

#carregar e prever valores da imagens
for img_path in img_paths:
  
    img = image.load_img(img_path, target_size=(28, 28))
    
  
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  
    img_array /= 255.0 

   
    prediction = model.predict(img_array)
    
   
    predicted_class = np.argmax(prediction)
    
   
    predictions.append(predicted_class)



for i, img_path in enumerate(img_paths):
    print(f"Imagem: {img_path}, Classe prevista: {predictions[i]}")




n = -1
i = 0
n1 = ''
n2 = ''

#loop para pegar dígitos até chegar no símbolo de operação
while n <=10:
    n1 = n1 + str(predictions[i])
    n = predictions[i+1]
    i +=1


i +=1
#loop para pegar dígitos depois do símbolo de operação
for j in range(i, len(predictions)):  
    n2 += str(predictions[j])
    if predictions[j] >= 10:
        n = -1  
        break 


n1 = int(n1)
n2 = int(n2)
op = n



#dicionário para definir operação
operations = {
    10: lambda a, b: a + b,
    11: lambda a, b: a / b if b != 0 else "Divisão por zero",
    12: lambda a, b: a == b,
    13: lambda a, b: a * b,
    14: lambda a, b: a - b
}
   
n3 = operations.get(op, lambda a, b: "Operação inválida")(n1, n2)


print(f'resultado: {n3}' )




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Imagem: ../test/a1.png, Classe prevista: 6
Imagem: ../test/a2.png, Classe prevista: 8
Imagem: ../test/a3.png, Classe prevista: 9
Imagem: ../test/b.png, Classe prevista: 14
Imagem: ../test/c1.png, Classe prevista: 2
Imagem: ../test/c2.png, Classe prevista: 3
Imagem: ../test/c3.png, Classe prevista: 7
resultado: 452
